In [1]:
import numpy as np
from datetime import timedelta
import datetime
import math
import time

첫번째로 일반적인 스텝다운 낙인형 ELS 상품을 계산하는 코드를 짜보았습니다.

이후에는 자산의 개수를 늘려보는 시도를 해보았고, 월지급식 스텝다운형을 만들기도 하였습니다.

마지막으로는 낙인 형태가 아니고 손실폭이 제한된 ELS 상품을 pricing하는 것으로 마무리를 하였습니다. 

코드는 내려가면서 점점 여러 형태의 ELS를 계산할 수 있는 일반적인 모델코드로 만들어보려고 시도하였습니다.

따라서 비슷해보이지만 점차 elif 문으로 추가가 되는 것을 보실 수 있습니다.

제가 여러 상품들을 코딩한 과정을 보여드리기 위해 코드들을 남겨두었습니다.

최종 코드는 맨 밑의 코드가 최종 코드라고 보시면 되겠습니다.

## 기본적인 스텝다운 낙인형 ELS 상품의 pricing 과정입니다.


In [2]:

#자산가격을 1로 설정하게끔하였습니다. 전부 만원 기준이기에 상관이 없다고 판단하였습니다.
S0 = 1

#중도 상환일
stop_day=["2022-05-31","2022-12-02","2023-06-01","2023-12-04","2024-06-03"]
#이자율은 이 부분에서만 수정해서 전체 모델이 적용이 되게끔 하였습니다. 이자율을 바꿔가며 모델을 돌려보았습니다.
r=0.023
mu=0.023
#일별 가격으로 3년치를 산출하기 위해 다음과 같이 설정하였습니다. / (1년 -> 365일)
T, N = 3, 365*3
dt=T/N

# k는 연 이율을 나눠주기 위한 변수입니다. / 상품마다 1년 내 상환주기가 다를 수 있으므로 바꿀 수 있게끔 설정하였습니다.
k=2 #6개월마다 중도 상환 시기가 있습니다.

profit=0.145 #연 수익률입니다.

#자산의 개수에 따라 변경할 수 있게끔 각 자산마다 변동성을 담길 수 있게끔 하였습니다.
sigma_list=[0.4,0.3579]
#knock-in 이 아닌 경우를 차후에 반영하기 위해서 knock_position을 설정하였습니다.
knock_position=1
#중도 상환일 마다의 기준치입니다.
stop_barrier=[0.85,0.85,0.85,0.85,0.8]

#기초자산의 개수입니다.
stock_num=2

#만기시 최소 수익 기준선과 낙인 기준선입니다.
final_line=0.75
knock_line=0.45
#covariance_matrix입니다. / 직접 설정을 해주셔야합니다.
cov_matrix=np.array([[1,0.47],[0.47,1]])
#날짜 시작 시기입니다.
start_point=datetime.datetime.strptime("2021-12-06", '%Y-%m-%d')

In [3]:
start=time.time()
price_si=[] # 각 시뮬레이션의 현재 가치를 넣어줄 리스트





for n in range(10000): #시뮬레이션 횟수 우선은 가격을 가볍게 알기 위해서 횟수를 적게하였습니다.
    rand_list=[]
    
    #처음 자산을 만들어주는 과정
    Spath = np.zeros(shape=N+1)
    Spath[0] = S0
    
    
    z_list= np.random.standard_normal((stock_num,N+1))
    
    
    #Covariance matrix를 분해한 값을 넣어주는 과정

    
    L=np.linalg.cholesky(cov_matrix)

    z_list=L@z_list # 상관계수 곱한 매트릭스
    
    
    
    
    
    path_new=[]
    #이후 기초자산의 개수에 따라 추가적으로 더 matrix를 형성 (주식개수 , 자산 path) /(2,356*4) 형태의 메트릭스 형성하였습니다.
    
    for z_n in range(stock_num): #stock 개수
        Spath1 = np.zeros(shape=N+1)
        Spath1[0] = S0
        for i in range(1, N+1): #다른 자산들의 path 
            #print(z_n)
            #z = np.random.standard_normal()
            Spath1[i] = Spath1[i-1] * np.exp((mu - 0.5 * sigma_list[z_n]**2) * dt + sigma_list[z_n] * np.sqrt(dt) * z_list[z_n][i-1])
        
        if path_new ==[]:
            path_new=Spath1
        else:
            path_new=np.vstack([path_new,Spath1])

    
    #시뮬레이션마다 중도 상환일을 하나씩 체크해줍니다. 
    out=0
    for index,i in enumerate(stop_day):
        first_time = datetime.datetime.strptime(i, '%Y-%m-%d')
        num= int(str(first_time-start_point)[:3])
        
        stop_price=np.round(path_new[:,num],2)
        if np.min(stop_price)>=stop_barrier[index]: #모두 기준선 안에 들어왔을경우, 수익
            
            
            price_si.append((1+(profit/k)*(1+index))/np.exp(r*num/365))

            out=1
            break

            
    if out==1: #만기까지 가지 않고 중도 상환할 경우 시뮬레이션 경우를 탈출합니다.
        out=0
        continue
    
    ################## 만기시#################
    
    
    
    if knock_position==1:
        last_time = datetime.datetime.strptime("2024-12-06", '%Y-%m-%d')
        num= int(str(last_time-start_point)[:4])
        
        if np.min(path_new[:,-1])>=final_line:
            price_si.append((1+(profit/k)*(len(stop_day)+1))/np.exp(r*num/365))
            continue
            
        # 만기시 만기기준가격 이상이 아니더라도 투자기간동안 항상 배리어가격보다 이상이면 수익입니다.
        elif np.min(path_new[:,:])>=knock_line :
            
            price_si.append((1+(profit/k)*(len(stop_day)+1))/np.exp(r*num/365))
            continue
        
        # 투자기간동안 한번이라도 배리어가격 이하일경우 손실을 추가할 수 있게끔 min 함수를 이용하여 값을 구한 뒤 넣어줍니다.
        
        elif np.min(path_new[:,:])<knock_line:
            print("knoxk",np.min(path_new[:,-1]))
            price_si.append(np.min(path_new[:,-1])/np.exp(r*num/365))
            continue
            
    
        
        
    
    
    
    
    
end=time.time()


<ipython-input-3-36c3b131be55>:41: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if path_new ==[]:


knoxk 0.3546048344386316
knoxk 0.21244034922012658
knoxk 0.2023137745703735
knoxk 0.1246185446184976
knoxk 0.48325000909182353
knoxk 0.4701167375097783
knoxk 0.2729228716735068
knoxk 0.19120827272391325
knoxk 0.515146598035748
knoxk 0.3383152013966407
knoxk 0.3785011977671403
knoxk 0.16222572748679814
knoxk 0.15851813574689197
knoxk 0.7073869208381629
knoxk 0.25395033848629406
knoxk 0.14695960109110306
knoxk 0.665688406152258
knoxk 0.2758603611771132
knoxk 0.27382245525631566
knoxk 0.4509082308935005
knoxk 0.30719577867451614
knoxk 0.05486073374734089
knoxk 0.42802139415958695
knoxk 0.4786140081931074
knoxk 0.22901345413123325
knoxk 0.48305309292288484
knoxk 0.6870971487042915
knoxk 0.645370024500986
knoxk 0.6094716607669759
knoxk 0.7188659636714512
knoxk 0.3917061271293042
knoxk 0.19603986032421136
knoxk 0.3128238001155632
knoxk 0.24065341672523238
knoxk 0.22691166833169304
knoxk 0.5915200742176806
knoxk 0.4943061381112073
knoxk 0.36389494334711237
knoxk 0.17259534395955475
knoxk 0.41

knoxk 0.4051932174135022
knoxk 0.6273114374031693
knoxk 0.3884900965113384
knoxk 0.4424560222723476
knoxk 0.4980764957202019
knoxk 0.43462749142904983
knoxk 0.5088641524828383
knoxk 0.5502339605437946
knoxk 0.45374563638684967
knoxk 0.1354842558544389
knoxk 0.2387984337183047
knoxk 0.43920265034527906
knoxk 0.14869028338657891
knoxk 0.26736632043897424
knoxk 0.15229628332735556
knoxk 0.3928540802537016
knoxk 0.3950110450784286
knoxk 0.2645709872059812
knoxk 0.14399215487802366
knoxk 0.22709330518334744
knoxk 0.409700646739925
knoxk 0.38267023632953273
knoxk 0.47673584638584354
knoxk 0.4235845576431028
knoxk 0.7199844580282101
knoxk 0.3105316368200242
knoxk 0.31655304216055785
knoxk 0.34680723064632546
knoxk 0.445414340458353
knoxk 0.36762034533529553
knoxk 0.21402897571510818
knoxk 0.2925259537883645
knoxk 0.17750757594121314
knoxk 0.5323997939624021
knoxk 0.4827743439071002
knoxk 0.3126564404971557
knoxk 0.3231044428589823
knoxk 0.3653943152805764
knoxk 0.5124791746110053
knoxk 0.3954

knoxk 0.3634634687189818
knoxk 0.6501526724816235
knoxk 0.373220222835335
knoxk 0.44787214725709346
knoxk 0.3528103817244259
knoxk 0.439891062601197
knoxk 0.4467181306786986
knoxk 0.24306195784369686
knoxk 0.2336252069724575
knoxk 0.39542777451595434
knoxk 0.6249262062356687
knoxk 0.3473610894220968
knoxk 0.6318126961064943
knoxk 0.32445192601195766
knoxk 0.5227459697325163
knoxk 0.2663142929463761
knoxk 0.25886278377341
knoxk 0.24999327843826252
knoxk 0.6776754044925017
knoxk 0.49892136653040714
knoxk 0.44981073195051174
knoxk 0.6058816151545224
knoxk 0.3619818703023772
knoxk 0.5225570301708187
knoxk 0.6467477713161213
knoxk 0.2201454645466437
knoxk 0.27411420765806016
knoxk 0.2003472847458704
knoxk 0.1998575646006435
knoxk 0.35500450962011504
knoxk 0.699591206734905
knoxk 0.14982024932232474
knoxk 0.1619023157834132
knoxk 0.3895248378626353
knoxk 0.4376934607331152
knoxk 0.42716639098046066
knoxk 0.34926811553669257
knoxk 0.47665566144887345
knoxk 0.43752823384143286
knoxk 0.33841491

knoxk 0.5833608227008938
knoxk 0.6986153909835536
knoxk 0.44266384867704395
knoxk 0.5317191297773075
knoxk 0.18720280046298238
knoxk 0.5081228548039793
knoxk 0.23671646895146983
knoxk 0.5624865993924028
knoxk 0.49818305450000766
knoxk 0.7319862494014975
knoxk 0.30031988443733254
knoxk 0.2479292144685273
knoxk 0.37100863855281924
knoxk 0.3103412723973914
knoxk 0.3211592244906864
knoxk 0.39637845236386476
knoxk 0.7003732560275063
knoxk 0.23847078039845154
knoxk 0.28348300786075126
knoxk 0.25198457531215057
knoxk 0.17896660831372088
knoxk 0.7448477068996349
knoxk 0.43321895024001744
knoxk 0.28726676891216785
knoxk 0.32212367667591785
knoxk 0.2093059098440747
knoxk 0.3512339672311214
knoxk 0.29923689308088
knoxk 0.2363207332017837
knoxk 0.3362307651932321
knoxk 0.3998394383928939
knoxk 0.5709994487508505
knoxk 0.5530940038181837
knoxk 0.5141015333608401
knoxk 0.4042750661243413
knoxk 0.4837124449590612
knoxk 0.42413933454793074
knoxk 0.4060795428694545
knoxk 0.2712229132213624
knoxk 0.4575

knoxk 0.4292946470038015
knoxk 0.2967814664848651
knoxk 0.5379142560886471
knoxk 0.5361946270546654
knoxk 0.08746814598735327
knoxk 0.44087124346513473
knoxk 0.34532955247418756
knoxk 0.3866149425906775
knoxk 0.29827594714078665
knoxk 0.6728075245478659
knoxk 0.25208352407901075
knoxk 0.3106490256679974
knoxk 0.3350831475383078
knoxk 0.6761101497716644
knoxk 0.5318966011197249
knoxk 0.5069452842725868
knoxk 0.4536241474398755
knoxk 0.4167029104283586
knoxk 0.6135932817324209
knoxk 0.5460355679281825
knoxk 0.3651471063537091
knoxk 0.3722913635941373
knoxk 0.4388625328528347
knoxk 0.7457620452672015
knoxk 0.7126900146539689
knoxk 0.11149191673300558
knoxk 0.5383260586648019
knoxk 0.4333432466230524
knoxk 0.16247287243080588
knoxk 0.3997518542817738
knoxk 0.6028215206947412
knoxk 0.3351608918042882
knoxk 0.34941731923928204
knoxk 0.5147867491791586
knoxk 0.3779154330002496
knoxk 0.5380160631562002
knoxk 0.40533160936525847
knoxk 0.48056562527299845
knoxk 0.24345079097536032
knoxk 0.233274

knoxk 0.5919120895302338
knoxk 0.22804655634756182
knoxk 0.25405390462198496
knoxk 0.673861937234188
knoxk 0.5484803142644034
knoxk 0.3293248787998865
knoxk 0.26734496500290994
knoxk 0.19959103012890175
knoxk 0.33520664483824036
knoxk 0.1924541647250521
knoxk 0.3799599169368405
knoxk 0.47056426933048673
knoxk 0.3019508009358591
knoxk 0.4787440165665228
knoxk 0.4575217165878004
knoxk 0.19741529121115559
knoxk 0.4469876141509105
knoxk 0.35118365501024656
knoxk 0.3105161245492141
knoxk 0.4604600574320132
knoxk 0.29543421656408975
knoxk 0.14735537167512952
knoxk 0.38421534740335056
knoxk 0.5627397811609924
knoxk 0.3786347613528401
knoxk 0.37109434715362327
knoxk 0.5635952084678689
knoxk 0.4504339534000764
knoxk 0.4248134705653977
knoxk 0.4985010517643304
knoxk 0.33508502619940317
knoxk 0.4459251383363631
knoxk 0.35141569668991307
knoxk 0.17138116070648055
knoxk 0.45966293262300356
knoxk 0.2652256559501736
knoxk 0.2544535434030192
knoxk 0.3555686350923152
knoxk 0.21427291565428785
knoxk 0.0

knoxk 0.6630487491939632
knoxk 0.45544889022929536
knoxk 0.21642638282573343
knoxk 0.35758055570160685
knoxk 0.509853979147015
knoxk 0.2951010452195821
knoxk 0.39253452669850275
knoxk 0.30694789879972284
knoxk 0.26976921381093255
knoxk 0.32069650516230924
knoxk 0.3293604921158391
knoxk 0.2415782532143116
knoxk 0.42675623933591594
knoxk 0.411094650702277
knoxk 0.37875062789091407
knoxk 0.2084261107932878
knoxk 0.31508259292212054
knoxk 0.3968130512087852
knoxk 0.285957625669417
knoxk 0.4394154889563741
knoxk 0.4033529443499455
knoxk 0.40287183710128704
knoxk 0.27106221969225525
knoxk 0.4764300098674808
knoxk 0.2053946430363199
knoxk 0.3218855817565024
knoxk 0.4463894388869276
knoxk 0.2404029497928139
knoxk 0.6118241294508735
knoxk 0.5606897936967654
knoxk 0.4152127315652721
knoxk 0.29786019428573524
knoxk 0.3711475058206992
knoxk 0.2103260121885994
knoxk 0.2502849364272053
knoxk 0.3390345429295209
knoxk 0.6461588762321628
knoxk 0.48027700911663723
knoxk 0.48884506280790074
knoxk 0.64822

In [4]:
sec=end-start
result = datetime.timedelta(seconds=sec) 
print(result) 
result_list = str(datetime.timedelta(seconds=sec)).split(".") 
print(result_list[0])


0:01:08.075637
0:01:08


In [5]:
print("계산결과값은 " + str(round(np.average(price_si)*10000,0))+"원 입니다.")
print("실제가격은 9275원입니다.")


계산결과값은 9396.0원 입니다.
실제가격은 9275원입니다.


## 이번에는 기초자산이 3개인 경우를 시도해보았습니다. 
#밑에서 알 수 있듯이 N개로 늘릴 수 있습니다. 대신에 그에 맞게 처음에 변수를 잘 설정을 해줄 필요가 있습니다.

In [6]:
import math
import pandas as pd
price_si=[] # 각 시뮬레이션의 현재 가치를 넣어줄 리스트

#r=0.03
#mu=0.03
#일별 가격으로 4년치를 산출하기 위해 다음과 같이 설정하였습니다.
year_num=365
T, N = 3, year_num*3

dt=T/N
#자산의 개수에 따라 변경할 수 있게끔 각 자산마다 변동성을 담길 수 있게끔 하였습니다.
sigma_list=[0.2753,0.264,0.2696]
#중도 상환일
stop_day=["2022-05-31","2022-12-02","2023-06-01","2023-12-04","2024-06-03"]
#knock-in 이 아닌 경우를 차후에 반영하기 위해서 knock_position을 설정하였습니다.
knock_position=1
#중도 상환일 마다의 기준치 입니다.
stop_barrier=[0.85,0.85,0.85,0.80,0.8]
#기초자산의 개수
stock_num=3
# k는 연 이율을 나눠주기 위한 변수입니다.
k=2
profit=0.051



cov_matrix=np.array([[1,0.1266,0.5949],[0.1266,1,0.3054],[0.5949,0.3054,1]])


#만기시 최소 수익 기준선과 낙인 기준선입니다.
final_line=0.8
knock_line=0.45
#날짜 시작 시기입니다.
start_point=datetime.datetime.strptime("2021-12-06", '%Y-%m-%d')




start=time.time()
price_si=[] # 각 시뮬레이션의 현재 가치를 넣어줄 리스트






for n in range(10000): #시뮬레이션 횟수 우선은 가격을 가볍게 알기 위해서 횟수를 적게하였습니다.
    rand_list=[]
    
    #처음 자산을 만들어주는 과정
    Spath = np.zeros(shape=N+1)
    Spath[0] = S0
    
    z_list= np.random.standard_normal((stock_num,N+1))
    #print(z_list)

    #Covariance matrix를 분해한 값을 넣어주는 과정

    
    L=np.linalg.cholesky(cov_matrix)

    z_list=L@z_list # 상관계수 곱한 매트릭스
    
    
    
    
    
    path_new=[]
    
    #이후 기초자산의 개수에 따라 추가적으로 더 matrix를 형성 (주식개수 , 자산 path) /(2,356*4) 형태의 메트릭스 형성하였습니다.
    for z_n in range(stock_num): #stock 개수
        Spath1 = np.zeros(shape=N+1)
        Spath1[0] = S0
        for i in range(1, N+1): #다른 자산들의 path 
            Spath1[i] = Spath1[i-1] * np.exp((mu - 0.5 * sigma_list[z_n]**2) * dt + sigma_list[z_n] * np.sqrt(dt) * z_list[z_n][i-1])
        
        if path_new ==[]:
            path_new=Spath1
        else:
            path_new=np.vstack([path_new,Spath1])

    #시뮬레이션마다 중도 상환일을 하나씩 체크해줍니다. 
    out=0
    for index,i in enumerate(stop_day):
        first_time = datetime.datetime.strptime(i, '%Y-%m-%d')
        num= int(str(first_time-start_point)[:4])
        stop_price=np.round(path_new[:,num],2)
        
        if np.min(stop_price)>=stop_barrier[index]: #모두 기준선 안에 들어왔을경우, 연 14.5% 수익
            
            
            price_si.append((1+(profit/k)*(1+index))/np.exp(r*num/365))

            out=1
            break

            
    if out==1: #만기까지 가지 않고 중도 상환할 경우 시뮬레이션 경우를 탈출합니다.
        out=0
        continue
    
    ################## 만기시#################
    
    
    
    if knock_position==1:
        last_time = datetime.datetime.strptime("2024-12-06", '%Y-%m-%d')
        num= int(str(last_time-start_point)[:4])
        # 만기시 만기기준가격 이상이면 수익입니다.
        if np.min(path_new[:,-1])>=final_line:
            price_si.append((1+(profit/k)*(len(stop_day)+1))/np.exp(r*num/365))
            continue
            
        # 만기시 만기기준가격 이상이 아니더라도 투자기간동안 항상 배리어가격보다 이상이면 수익입니다.
        elif np.min(path_new[:,:])>=knock_line :
            
            price_si.append((1+(profit/k)*(len(stop_day)+1))/np.exp(r*num/365))
            continue
        
        # 투자기간동안 한번이라도 배리어가격 이하일경우 손실을 추가할 수 있게끔 min 함수를 이용하여 값을 구한 뒤 넣어줍니다.
        
        elif np.min(path_new[:,:])<knock_line :
            print("knoxk",np.min(path_new[:,-1]))
            price_si.append(np.min(path_new[:,-1])/np.exp(r*num/365))
            continue
            
    
        
        
    
    
    
    
    
end=time.time()


<ipython-input-6-05c9055a8b1c>:75: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if path_new ==[]:


knoxk 0.5797381538247569
knoxk 0.5166053564381262
knoxk 0.40454153688684985
knoxk 0.4917949501254307
knoxk 0.40134185565761155
knoxk 0.4527245711330568
knoxk 0.4877911970027009
knoxk 0.5801063649971759
knoxk 0.3800573928090681
knoxk 0.4679619174013792
knoxk 0.2132603067043006
knoxk 0.31034744081659443
knoxk 0.3733145920747805
knoxk 0.5341631623157004
knoxk 0.4822560221835082
knoxk 0.3656613275479777
knoxk 0.4337911144818308
knoxk 0.48760357021027845
knoxk 0.44859766632855913
knoxk 0.3563599372307792
knoxk 0.3708224307199079
knoxk 0.24778334887039064
knoxk 0.40095193221911485
knoxk 0.3838184047190477
knoxk 0.6458912925017846
knoxk 0.6450430586300938
knoxk 0.49728475447184484
knoxk 0.3639815212796222
knoxk 0.3818793397838953
knoxk 0.469485868406739
knoxk 0.39811421024706933
knoxk 0.4009269174689622
knoxk 0.4392960382789049
knoxk 0.4468043109792387
knoxk 0.6076000795462216
knoxk 0.3733018199377804
knoxk 0.5316739269254724
knoxk 0.33440584178549654
knoxk 0.6015491085705144
knoxk 0.51498775

knoxk 0.3868323425396036
knoxk 0.4272677970050077
knoxk 0.4848135387173675
knoxk 0.5684411771941255
knoxk 0.44134344970255684
knoxk 0.4360379268477138
knoxk 0.4883657689857166
knoxk 0.5004007910991047
knoxk 0.3544466435017222
knoxk 0.36144072498075364
knoxk 0.5812515868121337
knoxk 0.41303258267698495
knoxk 0.5915464819937051
knoxk 0.47575620429450016
knoxk 0.3409518199038785
knoxk 0.3337911042706277
knoxk 0.5234266261930871
knoxk 0.3818705414970511
knoxk 0.5352842290318776
knoxk 0.3662014749136509
knoxk 0.4596147827468682
knoxk 0.3898734451733746
knoxk 0.4628987634007379
knoxk 0.37546655723100514
knoxk 0.2748236942715801
knoxk 0.6534674898579067
knoxk 0.5085834426985903
knoxk 0.3418828212238318
knoxk 0.3888767370804693
knoxk 0.4288471791796891
knoxk 0.4018999477997488
knoxk 0.3616968116612757
knoxk 0.36817471617077796
knoxk 0.621225065189584
knoxk 0.44140578156917165
knoxk 0.5785399497176572
knoxk 0.5373959957862456
knoxk 0.4786136270263977
knoxk 0.3984802742508962
knoxk 0.33378421581

knoxk 0.5890991419289171
knoxk 0.49130166396706654
knoxk 0.6473023574467638
knoxk 0.18637131024398923
knoxk 0.4618323257228254
knoxk 0.5651205343741755
knoxk 0.5166962183366784
knoxk 0.37959977796414224
knoxk 0.43997416930673877
knoxk 0.37832180622589834
knoxk 0.35895100177587513
knoxk 0.3839529956297218
knoxk 0.2805519065259232
knoxk 0.36712552461241593
knoxk 0.5928928115771249
knoxk 0.410940035480726
knoxk 0.3600535588722334
knoxk 0.5151494947682065
knoxk 0.20708897714812646
knoxk 0.5105285994957982
knoxk 0.49124590663227025
knoxk 0.4312423436496042
knoxk 0.2898107891517258
knoxk 0.2651992344186428
knoxk 0.5614144205440673
knoxk 0.29856138785595027
knoxk 0.3213256962068727
knoxk 0.28994367442409297
knoxk 0.26662936249456376
knoxk 0.42176574935028327
knoxk 0.4536001964580976
knoxk 0.5021206916709406
knoxk 0.4656084195701278
knoxk 0.4172763150518476
knoxk 0.6964110836260422
knoxk 0.3904269666608619
knoxk 0.19348743138515748
knoxk 0.44133502894819077
knoxk 0.3477309945590252
knoxk 0.552

knoxk 0.2639526854791745
knoxk 0.30393173788522987
knoxk 0.45937261104931604
knoxk 0.43774745793496445
knoxk 0.46113481864748035
knoxk 0.4274160486296138
knoxk 0.38552360528337515
knoxk 0.5896926470491353
knoxk 0.3593989794982741
knoxk 0.3582813739958042
knoxk 0.4761778773568349
knoxk 0.3099504662766006
knoxk 0.4154852689763103
knoxk 0.5245182296925576
knoxk 0.5711459431187687
knoxk 0.5918375651352861
knoxk 0.6027715877243681
knoxk 0.34691130183055957
knoxk 0.37094926924466576
knoxk 0.7612572304671259
knoxk 0.524503143889634
knoxk 0.3088327471872875
knoxk 0.34993815049146815
knoxk 0.393365700930253
knoxk 0.5073187342914947
knoxk 0.5629680381524378
knoxk 0.5423402011688426
knoxk 0.4484283700945141
knoxk 0.4888481013457377
knoxk 0.455295788889077
knoxk 0.34413859171384215
knoxk 0.3695390292868908
knoxk 0.4029112480943079
knoxk 0.4359553481369427
knoxk 0.2572090657440564
knoxk 0.28013695928195304
knoxk 0.5357449378287319
knoxk 0.37661942100804535
knoxk 0.3050915725038517
knoxk 0.362579631

In [7]:
print("계산결과값은 " + str(round(np.average(price_si)*10000,0))+"원 입니다.")
print("실제가격은 9039.68원입니다.")


계산결과값은 9476.0원 입니다.
실제가격은 9039.68원입니다.


In [8]:
sec=end-start
result = datetime.timedelta(seconds=sec) 
print(result) 
result_list = str(datetime.timedelta(seconds=sec)).split(".") 
print(result_list[0])


0:01:38.878214
0:01:38


# 녹인 형태가 아닌 스텝다운형 ELS (최대 손실폭 제한)

녹인 형태가 아닐경우 knock_position=0 을 통해 스텝다운만 계산되게끔 해주었습니다.

In [9]:
#자산가격을 1로 설정하게끔하였습니다. 전부 만원 기준이기에 상관이 없다고 판단하였습니다.
S0 = 1

#중도 상환일
stop_day=["2022-03-03","2022-05-31","2022-09-02"]
#일별 가격으로 3년치를 산출하기 위해 다음과 같이 설정하였습니다.
T, N = 1, 365*1
dt=T/N
monthly_pay=0
month_line=0

# k는 연 이율을 나눠주기 위한 변수입니다. / 상품마다 1년 내 상환주기가 다를 수 있으므로 바꿀 수 있게끔 설정하였습니다.
k=4

profit=0.134 #연 수익률입니다.

#자산의 개수에 따라 변경할 수 있게끔 각 자산마다 변동성을 담길 수 있게끔 하였습니다.
sigma_list=[0.2,0.2145]
#knock-in 이 아닌 경우를 차후에 반영하기 위해서 knock_position을 설정하였습니다.
knock_position=0
#중도 상환일 마다의 기준치입니다.
stop_barrier=[0.95,0.95,0.95]

#기초자산의 개수입니다.
stock_num=2

#만기시 최소 수익 기준선과 낙인 기준선입니다.
final_line=1

knock_line=0

max_loss_line= 0.85
#covariance_matrix입니다.
cov_matrix=np.array([[1,0.0058],[0.0058,1]])
#날짜 시작 시기입니다.
start_point=datetime.datetime.strptime("2021-12-06", '%Y-%m-%d')

In [10]:


start=time.time()
price_si=[] # 각 시뮬레이션의 현재 가치를 넣어줄 리스트


#start_point=datetime.datetime.strptime("2021-12-03", '%Y-%m-%d')




for n in range(10000): #시뮬레이션 횟수 우선은 가격을 가볍게 알기 위해서 횟수를 적게하였습니다.
    rand_list=[]
    
    #처음 자산을 만들어주는 과정
    Spath = np.zeros(shape=N+1)
    Spath[0] = S0
    
    z_list= np.random.standard_normal((stock_num,N+1))
    #print(z_list)

    #Covariance matrix를 분해한 값을 넣어주는 과정

    
    L=np.linalg.cholesky(cov_matrix)

    #print(L)
    #print(np.shape(z_list))
    z_list=L@z_list # 상관계수 곱한 매트릭스
    
    
    
    
    
    path_new=[]
    #이후 기초자산의 개수에 따라 추가적으로 더 matrix를 형성 (주식개수 , 자산 path) /(2,356*4) 형태의 메트릭스 형성하였습니다.
    for z_n in range(stock_num): #stock 개수
        Spath1 = np.zeros(shape=N+1)
        Spath1[0] = S0
        for i in range(1, N+1): #다른 자산들의 path 
            #print(z_n)
            #z = np.random.standard_normal()
            Spath1[i] = Spath1[i-1] * np.exp((mu - 0.5 * sigma_list[z_n]**2) * dt + sigma_list[z_n] * np.sqrt(dt) * z_list[z_n][i-1])
        
        if path_new ==[]:
            path_new=Spath1
        else:
            path_new=np.vstack([path_new,Spath1])

    #print(path_new)
    
    #시뮬레이션마다 중도 상환일을 하나씩 체크해줍니다. 
    out=0
    for index,i in enumerate(stop_day):
        first_time = datetime.datetime.strptime(i, '%Y-%m-%d')
        #print((first_time-start_point))
        num= int(str(first_time-start_point)[:3])
        
        stop_price=np.round(path_new[:,num+1],2)
        if np.all(stop_price>=stop_barrier[index])==True: #모두 기준선 안에 들어왔을경우, 연 14.5% 수익
            
            # 월지급식을 만든 뒤 노낙인 상품을 만들어서 월지급식 코드가 추가되어있습니다. 무시하셔도 되겠습니다.
            if monthly_pay==1:
                pay=0
                for day in range(6*index):
                    
                    month_time = datetime.datetime.strptime(month_list[day], '%Y-%m-%d')
                    num= int(str(month_time-start_point)[:3])
                    if np.all(path_new[:,num+1]>month_line)==True:
                        pay+=S0*0.0058/np.exp(r*num/365)
                        
                price_si.append(   (S0/ np.exp(r*num/365)) + pay )
                out=1
                break
            
            
            
            
            else:
                price_si.append((1+(profit/k)*(1+index))/np.exp(r*num/365))

                out=1
                break

            
    if out==1: #만기까지 가지 않고 중도 상환할 경우 시뮬레이션 경우를 탈출합니다.
        out=0
        continue
    
    ################## 만기시#################
    
    
    
    if knock_position==1 and monthly_pay==1:
        pay=0
        for day in month_list:
            
            month_time = datetime.datetime.strptime(day, '%Y-%m-%d')
            num= int(str(month_time-start_point)[:3])
            if np.all(path_new[:,num]>month_line)==True :
                pay+=S0*0.00585/np.exp(r*num/365)
        
        #2024년 12월 6일이 만기 일정입니다. / 초기에 변수를 설정하게끔 하였습니다.
        
        num= int(str(last_time-start_point)[:4])

        
        
        # 만기시 만기기준가격 이상이면 액면가 지급 
        if np.all(np.round(path_new[:,-1],2)>=final_line)==True:
            price_si.append( (S0/np.exp(r*num/365)) +pay)
            continue
            
        # 만기시 만기기준가격 이상이 아니더라도 투자기간동안 항상 배리어가격보다 이상이면 액면가지급
        elif np.all(np.round(path_new[:,:],2)>=knock_line) ==True :
            
            #print(1+(0.145/2)*(len(stop_day)+1)/np.exp(r*num/360))
            #print(num/360)
            #print(np.exp(r*num/360))
            price_si.append((S0/np.exp(r*num/365))+pay)
            continue
        
        # 투자기간동안 한번이라도 배리어가격 이하일경우 손실을 추가할 수 있게끔 min 함수를 이용하여 값을 구한 뒤 넣어줍니다.
        #손실을 입어도 중도에 월지급을 생각해주어야합니다. 따라서 pay를 더해주었습니다.
        elif np.any(np.round(path_new[:,:],2)<knock_line) ==True:
            print("knoxk",np.min(path_new[:,-1]))
            price_si.append( (np.min(path_new[:,-1])/np.exp(r*num/365)) +pay )
            continue
        
        
        
    
    
    
        
        
    
    
    
    
    
    
    elif knock_position==1:
        last_time = datetime.datetime.strptime("2024-12-06", '%Y-%m-%d')
        num= int(str(last_time-start_point)[:4])
        
        if np.min(path_new[:,-1])>=final_line:
            price_si.append((1+(profit/k)*(len(stop_day)+1))/np.exp(r*num/365))
            continue
            
        # 만기시 만기기준가격 이상이 아니더라도 투자기간동안 항상 배리어가격보다 이상이면 수익입니다.
        elif np.min(path_new[:,:])>=knock_line :
            
            price_si.append((1+(profit/k)*(len(stop_day)+1))/np.exp(r*num/365))
            continue
        
        # 투자기간동안 한번이라도 배리어가격 이하일경우 손실을 추가할 수 있게끔 min 함수를 이용하여 값을 구한 뒤 넣어줍니다.
        
        elif np.min(path_new[:,:])<knock_line:
            print("knoxk",np.min(path_new[:,-1]))
            price_si.append(np.min(path_new[:,-1])/np.exp(r*num/365))
            continue
    
    
    elif knock_position==0: #노 낙인
        first_time = datetime.datetime.strptime("2022-11-30", '%Y-%m-%d')
        num= int(str(first_time-start_point)[:4])

        # 만기시 만기기준가격 이상이면 수익입니다.
        
        if np.min(path_new[:,-1])>=final_line:
            price_si.append( np.min(path_new[:,-1])/np.exp(r*num/365))
            #continue
        
        else :
            if np.min(path_new[:,-1]) < max_loss_line:
                price_si.append(max_loss_line/np.exp(r*num/365))
            
            else:
                price_si.append( np.min(path_new[:,-1])/np.exp(r*num/365))
                

<ipython-input-10-b535e4d3d01b>:43: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if path_new ==[]:


In [11]:
print("계산결과값은 " + str(round(np.average(price_si)*10000,0))+"원 입니다.")
print("실제가격은 9601.10원입니다.")


계산결과값은 9758.0원 입니다.
실제가격은 9601.10원입니다.



## 월지급식 ELS


monthly_pay==1 로 설정해주셔야하고

month_line 변수는 월마다 자산가격이 넘어야하는 선입니다.

month_list에 월수익 지급일을 적어주셔야합니다. 월이 너무 많기에 저는 일수는 고정시키고 월만 이동시켜보았습니다.


In [12]:



S0 = 1

#중도 상환일
stop_day=["2022-05-31","2022-12-02","2023-06-01","2023-12-04","2024-06-03"]
month_list=["2022-01-04",'2022-01-28','2022-03-02','2022-04-04','2022-05-03','2022-05-31','2022-07-01','2022-08-03','2022-09-02',
           '2022-10-02','2022-11-02','2022-12-02',
            "2023-01-04",'2023-01-28','2023-03-02','2023-04-04','2023-05-03','2023-05-31','2023-07-01','2023-08-03','2023-09-02',
           '2023-10-02','2023-11-02','2023-12-02',
            "2024-01-04",'2024-01-28','2024-03-02','2024-04-04','2024-05-03','2024-05-31','2024-07-01','2024-08-03','2024-09-02',
           '2024-10-02','2024-11-02','2024-12-02',
           
           ]
monthly_pay=1
month_line=0.6

#일별 가격으로 4년치를 산출하기 위해 다음과 같이 설정하였습니다.
T, N = 3, 365*3
dt=T/N

# k는 연 이율을 나눠주기 위한 변수입니다. / 상품마다 1년 내 상환주기가 다를 수 있으므로 바꿀 수 있게끔 설정하였습니다.
k=2

profit=0.145 #연 수익률입니다.

#자산의 개수에 따라 변경할 수 있게끔 각 자산마다 변동성을 담길 수 있게끔 하였습니다.
sigma_list=[0.21,0.2753]
#knock-in 이 아닌 경우를 차후에 반영하기 위해서 knock_position을 설정하였습니다.
knock_position=1
#중도 상환일 마다의 기준치입니다.
stop_barrier=[0.95,0.9,0.85,0.85,0.8]

#기초자산의 개수입니다.
stock_num=2

#만기시 최소 수익 기준선과 낙인 기준선입니다.
final_line=0.8
knock_line=0.5
#covariance_matrix입니다.
cov_matrix=np.array([[1,0.38],[0.38,1]])
#날짜 시작 시기입니다.
start_point=datetime.datetime.strptime("2021-12-06", '%Y-%m-%d')
last_time = datetime.datetime.strptime('2024-12-06', '%Y-%m-%d')

In [13]:
start=time.time()
price_si=[] # 각 시뮬레이션의 현재 가치를 넣어줄 리스트





for n in range(10000): #시뮬레이션 횟수 우선은 가격을 가볍게 알기 위해서 횟수를 적게하였습니다.
    rand_list=[]
    
    #처음 자산을 만들어주는 과정
    Spath = np.zeros(shape=N+1)
    Spath[0] = S0
    
    z_list= np.random.standard_normal((stock_num,N+1))
    #print(z_list)

    #Covariance matrix를 분해한 값을 넣어주는 과정

    
    L=np.linalg.cholesky(cov_matrix)

    #print(L)
    #print(np.shape(z_list))
    z_list=L@z_list # 상관계수 곱한 매트릭스
    
    
    
    
    
    path_new=[]
    #이후 기초자산의 개수에 따라 추가적으로 더 matrix를 형성 (주식개수 , 자산 path) /(2,356*4) 형태의 메트릭스 형성하였습니다.
    for z_n in range(stock_num): #stock 개수
        Spath1 = np.zeros(shape=N+1)
        Spath1[0] = S0
        for i in range(1, N+1): #다른 자산들의 path 
            #print(z_n)
            #z = np.random.standard_normal()
            Spath1[i] = Spath1[i-1] * np.exp((mu - 0.5 * sigma_list[z_n]**2) * dt + sigma_list[z_n] * np.sqrt(dt) * z_list[z_n][i-1])
        
        if path_new ==[]:
            path_new=Spath1
        else:
            path_new=np.vstack([path_new,Spath1])

    
    #시뮬레이션마다 중도 상환일을 하나씩 체크해줍니다. 
    out=0
    for index,i in enumerate(stop_day):
        first_time = datetime.datetime.strptime(i, '%Y-%m-%d')
        #print((first_time-start_point))
        num= int(str(first_time-start_point)[:3])
        
        stop_price=np.round(path_new[:,num+1],2)
        if np.all(stop_price>=stop_barrier[index])==True: #모두 기준선 안에 들어왔을경우, mothly pay값에 따라 수익구조가 바뀝니다.
            
            
            if monthly_pay==1: #이 코드는 중도상환이 가능할때마다 월수익의 현재가치를 pay변수에 담아 액면가의 현재가치와 더한 
                pay=0          #현금흐름을 구하는 과정입니다.
                for day in range(6*index):
                    
                    month_time = datetime.datetime.strptime(month_list[day], '%Y-%m-%d')
                    num= int(str(month_time-start_point)[:3])
                    if np.all(path_new[:,num+1]>month_line)==True:
                        pay+=S0*0.0058/np.exp(r*num/365)
                        
                price_si.append(   (S0/ np.exp(r*num/365)) + pay )
                out=1
                break
            
            
            
            
            else:
                price_si.append((1+(profit/k)*(1+index))/np.exp(r*num/365))

                out=1
                break

            
    if out==1: #만기까지 가지 않고 중도 상환할 경우 시뮬레이션 경우를 탈출합니다.
        out=0
        continue
    
    ################## 만기시#################
    
    if knock_position==1 and monthly_pay==1:
        pay=0
        for day in month_list:
            
            month_time = datetime.datetime.strptime(day, '%Y-%m-%d')
            num= int(str(month_time-start_point)[:3])
            if np.all(path_new[:,num]>month_line)==True :
                pay+=S0*0.00585/np.exp(r*num/365)
        
        #2024년 12월 6일이 만기 일정입니다. / 초기에 변수를 설정하게끔 하였습니다.
        
        num= int(str(last_time-start_point)[:4])

        
        
        # 만기시 만기기준가격 이상이면 액면가 지급 
        if np.all(np.round(path_new[:,-1],2)>=final_line)==True:
            price_si.append( (S0/np.exp(r*num/365)) +pay)
            continue
            
        # 만기시 만기기준가격 이상이 아니더라도 투자기간동안 항상 배리어가격보다 이상이면 액면가지급
        elif np.all(np.round(path_new[:,:],2)>=knock_line) ==True :
            
            #print(1+(0.145/2)*(len(stop_day)+1)/np.exp(r*num/360))
            #print(num/360)
            #print(np.exp(r*num/360))
            price_si.append((S0/np.exp(r*num/365))+pay)
            continue
        
        # 투자기간동안 한번이라도 배리어가격 이하일경우 손실을 추가할 수 있게끔 min 함수를 이용하여 값을 구한 뒤 넣어줍니다.
        #손실을 입어도 중도에 월지급을 생각해주어야합니다. 따라서 pay를 더해주었습니다.
        elif np.any(np.round(path_new[:,:],2)<knock_line) ==True:
            print("knoxk",np.min(path_new[:,-1]))
            price_si.append( (np.min(path_new[:,-1])/np.exp(r*num/365)) +pay )
            continue
        
        
        
    
    
    
    
    
    
    elif knock_position==1:
        last_time = datetime.datetime.strptime("2024-12-06", '%Y-%m-%d')
        num= int(str(last_time-start_point)[:4])
        # 만기시 만기기준가격 이상이면 수익입니다.
        if np.min(path_new[:,-1])>=final_line:
            price_si.append((1+(profit/k)*(len(stop_day)+1))/np.exp(r*num/365))
            continue
            
        # 만기시 만기기준가격 이상이 아니더라도 투자기간동안 항상 배리어가격보다 이상이면 수익입니다.
        elif np.min(path_new[:,:])>=knock_line :
            
            price_si.append((1+(profit/k)*(len(stop_day)+1))/np.exp(r*num/365))
            continue
        
        # 투자기간동안 한번이라도 배리어가격 이하일경우 손실을 추가할 수 있게끔 min 함수를 이용하여 값을 구한 뒤 넣어줍니다.
        
        elif np.min(path_new[:,:])<knock_line :
            print("knoxk",np.min(path_new[:,-1]))
            price_si.append(np.min(path_new[:,-1])/np.exp(r*num/365))
            continue
            
    
        
        
    
    
    
    
    
end=time.time()

knoxk 0.4256089352175043
knoxk 0.36468742389334535
knoxk 0.40386035394497194


<ipython-input-13-522f757c03cc>:41: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if path_new ==[]:


knoxk 0.6496477261990993
knoxk 0.37894259112700723
knoxk 0.6613006496556407
knoxk 0.3370675191636539
knoxk 0.25348933246245625
knoxk 0.2890778456074156
knoxk 0.2801136255486864
knoxk 0.3027752448203685
knoxk 0.3812239303498694
knoxk 0.49764462550016575
knoxk 0.6634365544620799
knoxk 0.5306383958766041
knoxk 0.5285258101973149
knoxk 0.47476800639344563
knoxk 0.3942788266983989
knoxk 0.44549174411141224
knoxk 0.6553727403232904
knoxk 0.4142651326817053
knoxk 0.4711010857742353
knoxk 0.5749900657888047
knoxk 0.4697177721890127
knoxk 0.3316498119977528
knoxk 0.579580597701946
knoxk 0.5943110983847619
knoxk 0.4635866253984139
knoxk 0.48459505952521137
knoxk 0.5319885506207263
knoxk 0.6325023260221012
knoxk 0.35634271107573473
knoxk 0.41102757617500013
knoxk 0.4548072155484484
knoxk 0.23763656227830027
knoxk 0.2698283707823077
knoxk 0.28007083974771596
knoxk 0.5293855182865375
knoxk 0.45964380259827575
knoxk 0.6068571827454077
knoxk 0.5319079146443271
knoxk 0.7626869333528963
knoxk 0.2269379

knoxk 0.6103410480794803
knoxk 0.6488554276565387
knoxk 0.5593544153656806
knoxk 0.43611171778995467
knoxk 0.6164659481846025
knoxk 0.46561304698532435
knoxk 0.3443806262101873
knoxk 0.3812967025683023
knoxk 0.38494325989040984
knoxk 0.41245639718133087
knoxk 0.5702446026740539
knoxk 0.6044564075264508
knoxk 0.4458656966577966
knoxk 0.47767081131584116
knoxk 0.5143584042592233
knoxk 0.4630686250342095
knoxk 0.6034863973299178
knoxk 0.43034290202818126
knoxk 0.3805668425081738
knoxk 0.5197620111817346
knoxk 0.3046189604100225
knoxk 0.46383103377869966
knoxk 0.40314729928849674
knoxk 0.5432455292818908
knoxk 0.609929078949292
knoxk 0.44175617327197864
knoxk 0.7135515532088976
knoxk 0.5446600027785098
knoxk 0.413646439045944
knoxk 0.25293689340873454
knoxk 0.5160405857312479
knoxk 0.2729293414564493
knoxk 0.4468494448604485
knoxk 0.32040680648463665
knoxk 0.601992377652752
knoxk 0.34863096479430766
knoxk 0.5641103491995652
knoxk 0.5634210999640377
knoxk 0.5771426158375801
knoxk 0.41063223

knoxk 0.22655787809842015
knoxk 0.41138296930194473
knoxk 0.518474449954156
knoxk 0.48624424637351255
knoxk 0.6197324180325511
knoxk 0.46260157479218894
knoxk 0.26952551882129705
knoxk 0.5673241444819085
knoxk 0.5211061247800561
knoxk 0.3530099894585414
knoxk 0.4536765934244321
knoxk 0.3611854989244224
knoxk 0.36798015846980975
knoxk 0.4090051500776027
knoxk 0.41876704314448243
knoxk 0.6383013510403094
knoxk 0.6769454546872078
knoxk 0.5883147629972034
knoxk 0.41049747463534675
knoxk 0.7554664603105078
knoxk 0.39555407364866013
knoxk 0.564376818154422
knoxk 0.4296854078550632
knoxk 0.4546361997338132
knoxk 0.5545389682838338
knoxk 0.521404019530716
knoxk 0.36634702556148624
knoxk 0.36014279584996856
knoxk 0.4217698828830856
knoxk 0.6459705951821324
knoxk 0.47961023608616343
knoxk 0.38747800153097656
knoxk 0.4692453815715904
knoxk 0.4115173391175654
knoxk 0.6029379803116255
knoxk 0.4835617276028416
knoxk 0.5092270462500449
knoxk 0.4452924123830167
knoxk 0.3824737292562685
knoxk 0.5481454

knoxk 0.5305554281313039
knoxk 0.5114780860432623
knoxk 0.4932977787505259
knoxk 0.5013116875261497
knoxk 0.590525693696709
knoxk 0.5154200073062691
knoxk 0.4012321538222386
knoxk 0.5010341587759836
knoxk 0.46863664274799977
knoxk 0.6332148617573782
knoxk 0.5682959909037334
knoxk 0.5551821783343371
knoxk 0.5062087885342081
knoxk 0.47268375582604005
knoxk 0.30281975864758465
knoxk 0.28023395621032693
knoxk 0.29249527111547136
knoxk 0.4557900909154803
knoxk 0.3331434227739489
knoxk 0.42415162020323677
knoxk 0.5605694460635801
knoxk 0.5187688609112552
knoxk 0.32868153641231046
knoxk 0.4814501038650516
knoxk 0.5874505238259213
knoxk 0.36876540769458876
knoxk 0.4143249413400562
knoxk 0.501146131274187
knoxk 0.4593690557132363
knoxk 0.4053674471506095
knoxk 0.5706419459523987
knoxk 0.3147389480464883
knoxk 0.20898283785214053
knoxk 0.5584347830395517
knoxk 0.3860315955389827
knoxk 0.7308979749689603
knoxk 0.4954403672491486
knoxk 0.4844543250495597
knoxk 0.4695239706070874
knoxk 0.2263768131

In [14]:
print("계산결과값은 " + str(round(np.average(price_si)*10000,0))+"원 입니다.")
print("실제가격은 9665.68원입니다.")


계산결과값은 9697.0원 입니다.
실제가격은 9665.68원입니다.


In [15]:
sec=end-start
result = datetime.timedelta(seconds=sec) 
print(result) 
result_list = str(datetime.timedelta(seconds=sec)).split(".") 
print(result_list[0])


0:01:12.743245
0:01:12


## 멀티베리어 ELS pricing을 포함한 최종 코드

다음 코드는 제가 최종적으로 만든 코드입니다. 다양한 ELS상품들을 종합적으로 적정가를 계산할 수 있게끔 설계하였습니다.

In [16]:

#자산가격을 1로 설정하게끔하였습니다. 전부 만원 기준이기에 상관이 없다고 판단하였습니다.
S0 = 1

#중도 상환일
stop_day=["2022-05-31","2022-12-02","2023-06-01","2023-12-04","2024-06-03"]
#일별 가격으로 3년치를 산출하기 위해 다음과 같이 설정하였습니다.
T, N = 3, 365*3
dt=T/N

# k는 연 이율을 나눠주기 위한 변수입니다. / 상품마다 1년 내 상환주기가 다를 수 있으므로 바꿀 수 있게끔 설정하였습니다.
k=2

profit=0.09 #연 수익률입니다.

#자산의 개수에 따라 변경할 수 있게끔 각 자산마다 변동성을 담길 수 있게끔 하였습니다.
sigma_list=[0.4,0.3624,0.2924]
#knock-in 이 아닌 경우를 차후에 반영하기 위해서 knock_position을 설정하였습니다.
knock_position=1
#중도 상환일 마다의 기준치입니다.
stop_barrier=[0.92,0.90,0.87,0.82,0.8]

#기초자산의 개수입니다.
stock_num=3

#만기시 최소 수익 기준선과 낙인 기준선입니다.
final_line=0.75
knock_line=0.52
#covariance_matrix입니다. / 직접 설정을 해주셔야합니다.
cov_matrix=np.array([[1,0.3753,-0.0109],[0.3753,1,-0.027],[-0.0109,-0.027,1]])


#날짜 시작 시기입니다.
start_point=datetime.datetime.strptime("2021-12-06", '%Y-%m-%d')

ELS_name="multi_barrier" #월수익지급시 monthly_pay 로 이름을 바꿔주게끔 하였습니다.
multi_list=[1,2] #멀티배리어의 경우 몇번째 조기상환시점에 배리어가 있는지 리스트에 적어줍니다.
multi_barrier=[0.87 , 0.82] #각 멀티배리어 시점마다 배리어 라인의 % 입니다.


In [17]:
start=time.time()
price_si=[] # 각 시뮬레이션의 현재 가치를 넣어줄 리스트





for n in range(10000): #시뮬레이션 횟수 우선은 가격을 가볍게 알기 위해서 횟수를 적게하였습니다.
    rand_list=[]
    
    #처음 자산을 만들어주는 과정
    Spath = np.zeros(shape=N+1)
    Spath[0] = S0
    
    z_list= np.random.standard_normal((stock_num,N+1))
    #print(z_list)

    #Covariance matrix를 분해한 값을 넣어주는 과정

    
    L=np.linalg.cholesky(cov_matrix)

    #print(L)
    #print(np.shape(z_list))
    z_list=L@z_list # 상관계수 곱한 매트릭스
    
    
    
    
    
    path_new=[]
    #이후 기초자산의 개수에 따라 추가적으로 더 matrix를 형성 (주식개수 , 자산 path) /(2,356*4) 형태의 메트릭스 형성하였습니다.
    for z_n in range(stock_num): #stock 개수
        Spath1 = np.zeros(shape=N+1)
        Spath1[0] = S0
        for i in range(1, N+1): #다른 자산들의 path 
            #print(z_n)
            #z = np.random.standard_normal()
            Spath1[i] = Spath1[i-1] * np.exp((mu - 0.5 * sigma_list[z_n]**2) * dt + sigma_list[z_n] * np.sqrt(dt) * z_list[z_n][i-1])
        
        if path_new ==[]:
            path_new=Spath1
        else:
            path_new=np.vstack([path_new,Spath1])

    
    #시뮬레이션마다 중도 상환일을 하나씩 체크해줍니다. 
    out=0
    for index,i in enumerate(stop_day):
        first_time = datetime.datetime.strptime(i, '%Y-%m-%d')
        #print((first_time-start_point))
        num= int(str(first_time-start_point)[:3])
        
        stop_price=np.round(path_new[:,num+1],2)
        if np.all(stop_price>=stop_barrier[index])==True: #모두 기준선 안에 들어왔을경우, mothly pay값에 따라 수익구조가 바뀝니다.
            
            if ELS_name=="multi_barrier":
                
                if index+1 in multi_list:
                    if np.min(stop_price) >= multi_barrier[index] :
                        price_si.append((1+(profit/k)*(1+index))/np.exp(r*num/365))
                        break


            if ELS_name=="monthly_pay": #이 코드는 중도상환이 가능할때마다 월수익의 현재가치를 pay변수에 담아 액면가의 현재가치와 더한 
                pay=0          #현금흐름을 구하는 과정입니다.
                for day in range(6*index):
                    
                    month_time = datetime.datetime.strptime(month_list[day], '%Y-%m-%d')
                    num= int(str(month_time-start_point)[:3])
                    if np.all(path_new[:,num+1]>month_line)==True:
                        pay+=S0*0.0058/np.exp(r*num/365)
                        
                price_si.append(   (S0/ np.exp(r*num/365)) + pay )
                out=1
                break
            
            
            
            
            else:
                price_si.append((1+(profit/k)*(1+index))/np.exp(r*num/365))

                out=1
                break

            
    if out==1: #만기까지 가지 않고 중도 상환할 경우 시뮬레이션 경우를 탈출합니다.
        out=0
        continue
    
    ################## 만기시#################
    
    if knock_position==1 and ELS_name=="monthly_pay":
        pay=0
        for day in month_list:
            
            month_time = datetime.datetime.strptime(day, '%Y-%m-%d')
            num= int(str(month_time-start_point)[:3])
            if np.all(path_new[:,num]>month_line)==True :
                pay+=S0*0.00585/np.exp(r*num/365)
        
        #2024년 12월 6일이 만기 일정입니다. / 초기에 변수를 설정하게끔 하였습니다.
        
        num= int(str(last_time-start_point)[:4])

        
        
        # 만기시 만기기준가격 이상이면 액면가 지급 
        if np.all(np.round(path_new[:,-1],2)>=final_line)==True:
            price_si.append( (S0/np.exp(r*num/365)) +pay)
            continue
            
        # 만기시 만기기준가격 이상이 아니더라도 투자기간동안 항상 배리어가격보다 이상이면 액면가지급
        elif np.all(np.round(path_new[:,:],2)>=knock_line) ==True :
            
            #print(1+(0.145/2)*(len(stop_day)+1)/np.exp(r*num/360))
            #print(num/360)
            #print(np.exp(r*num/360))
            price_si.append((S0/np.exp(r*num/365))+pay)
            continue
        
        # 투자기간동안 한번이라도 배리어가격 이하일경우 손실을 추가할 수 있게끔 min 함수를 이용하여 값을 구한 뒤 넣어줍니다.
        #손실을 입어도 중도에 월지급을 생각해주어야합니다. 따라서 pay를 더해주었습니다.
        elif np.any(np.round(path_new[:,:],2)<knock_line) ==True:
            print("knoxk",np.min(path_new[:,-1]))
            price_si.append( (np.min(path_new[:,-1])/np.exp(r*num/365)) +pay )
            continue
        
        
        
    
    
    
    
    
    
    elif knock_position==1:
        last_time = datetime.datetime.strptime("2024-12-06", '%Y-%m-%d')
        num= int(str(last_time-start_point)[:4])
        # 만기시 만기기준가격 이상이면 수익입니다.
        if np.min(path_new[:,-1])>=final_line:
            price_si.append((1+(profit/k)*(len(stop_day)+1))/np.exp(r*num/365))
            continue
            
        # 만기시 만기기준가격 이상이 아니더라도 투자기간동안 항상 배리어가격보다 이상이면 수익입니다.
        elif np.min(path_new[:,:])>=knock_line :
            
            price_si.append((1+(profit/k)*(len(stop_day)+1))/np.exp(r*num/365))
            continue
        
        # 투자기간동안 한번이라도 배리어가격 이하일경우 손실을 추가할 수 있게끔 min 함수를 이용하여 값을 구한 뒤 넣어줍니다.
        
        elif np.min(path_new[:,:])<knock_line :
            print("knoxk",np.min(path_new[:,-1]))
            price_si.append(np.min(path_new[:,-1])/np.exp(r*num/365))
            continue
            
    
        
        
    
    
    
    
    
end=time.time()

knoxk 0.4457514787912167
knoxk 0.3453635853942901
knoxk 0.5021129820467932
knoxk 

<ipython-input-17-38778ac1c57c>:41: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if path_new ==[]:


0.5160458287932876
knoxk 0.19410576937436594
knoxk 0.340801809413006
knoxk 0.40920242359754494
knoxk 0.31102632522698603
knoxk 0.2619270535875272
knoxk 0.259363457639803
knoxk 0.42707047313634017
knoxk 0.34074681686042363
knoxk 0.45914934540495234
knoxk 0.3457635925945404
knoxk 0.3227465233158038
knoxk 0.6234914992050818
knoxk 0.6986637799663538
knoxk 0.3440716713735855
knoxk 0.5760773689801814
knoxk 0.5621817162073537
knoxk 0.40694241527712566
knoxk 0.17248074447708314
knoxk 0.3200064353621566
knoxk 0.2992446907280208
knoxk 0.3687716769459478
knoxk 0.07372196151761166
knoxk 0.22945392295485975
knoxk 0.5643273160006135
knoxk 0.5381931952286344
knoxk 0.5260869697390953
knoxk 0.3628556676701341
knoxk 0.4730613921658307
knoxk 0.20188444996222665
knoxk 0.6881553952288273
knoxk 0.5540694002395214
knoxk 0.3881582469412339
knoxk 0.6090317419423714
knoxk 0.3998771202637599
knoxk 0.4221934180386018
knoxk 0.5192464349045542
knoxk 0.22238663334428946
knoxk 0.16290126662592397
knoxk 0.237087196201

knoxk 0.4123956839826881
knoxk 0.16066151674163162
knoxk 0.5083151321141981
knoxk 0.4196532609589094
knoxk 0.2745130920916917
knoxk 0.38936068860021245
knoxk 0.35666109914904875
knoxk 0.3840019627417235
knoxk 0.3636281169208077
knoxk 0.42011581575764595
knoxk 0.36878857820100625
knoxk 0.382596481715971
knoxk 0.29814164116844416
knoxk 0.609796670568498
knoxk 0.5078065548673107
knoxk 0.2447336021495776
knoxk 0.31220462197801196
knoxk 0.45103912403255103
knoxk 0.17854707982994128
knoxk 0.33536756996006356
knoxk 0.3950220042890551
knoxk 0.4795013979916132
knoxk 0.5153487626325131
knoxk 0.23689078650642836
knoxk 0.4223219212813321
knoxk 0.5540828472709981
knoxk 0.2955429176804763
knoxk 0.47010080768841
knoxk 0.272026769435478
knoxk 0.5180532576035055
knoxk 0.2621754925054481
knoxk 0.504804032991912
knoxk 0.3782311684780136
knoxk 0.24060012560389582
knoxk 0.6186956419638914
knoxk 0.2335725838460293
knoxk 0.49786860851027137
knoxk 0.7102507630869737
knoxk 0.5443197191787504
knoxk 0.4440019325

knoxk 0.5398621925092261
knoxk 0.2855893079087108
knoxk 0.6925804037692577
knoxk 0.4502985365061131
knoxk 0.6012515776335213
knoxk 0.2948591434314951
knoxk 0.5606930709945116
knoxk 0.7357197354953227
knoxk 0.4277691661529753
knoxk 0.43778198680493025
knoxk 0.3571763777859705
knoxk 0.2600847967854488
knoxk 0.5332029568271693
knoxk 0.22802194425843963
knoxk 0.7316821513756528
knoxk 0.5628449649929954
knoxk 0.43535623320354566
knoxk 0.527713027032868
knoxk 0.17703316730193908
knoxk 0.33060139956771045
knoxk 0.3848785623798878
knoxk 0.4431485846852884
knoxk 0.2388855633569111
knoxk 0.4420571684579531
knoxk 0.44893866417281253
knoxk 0.5250092784111536
knoxk 0.12812204831869964
knoxk 0.4795764239480777
knoxk 0.5178007316174487
knoxk 0.640069466008654
knoxk 0.7301771787814888
knoxk 0.27333301904988044
knoxk 0.24747131158031282
knoxk 0.5235156154044305
knoxk 0.21862916728689052
knoxk 0.49152075752917923
knoxk 0.6419759343055446
knoxk 0.22365075648674518
knoxk 0.6983147294357831
knoxk 0.5818298

...

knoxk 0.3970779323140389
knoxk 0.5096166523654556
knoxk 0.49111757206011125
knoxk 0.25579717809368274
knoxk 0.5936359073153347
knoxk 0.5295918803491704
knoxk 0.18473162404457896
knoxk 0.37301459436198126
knoxk 0.36904999797683835
knoxk 0.5000975531862688
knoxk 0.5270365180067076
knoxk 0.7254905712705374
knoxk 0.6969811717401445
knoxk 0.2899235460735475
knoxk 0.4273704095822071
knoxk 0.47343989620690935
knoxk 0.5483631736155377
knoxk 0.5786301669892536
knoxk 0.2269217358695612
knoxk 0.7084518630303862
knoxk 0.32423523857247294
knoxk 0.44934556949387827
knoxk 0.5268200333898927
knoxk 0.47372828075617424
knoxk 0.5563003060999754
knoxk 0.4286493327679936
knoxk 0.34925080341607073
knoxk 0.42167584447838724
knoxk 0.49332916459466064
knoxk 0.42431041528809244
knoxk 0.4297043173212003
knoxk 0.5047977807792356
knoxk 0.2605740010105872
knoxk 0.5164094243984032
knoxk 0.4919306610427135
knoxk 0.48661822335053956
knoxk 0.39599608758434507
knoxk 0.22408600066967638
knoxk 0.6396201459224081
knoxk 0.4

knoxk 0.595181016080125
knoxk 0.27674576341959883
knoxk 0.20361575849804667
knoxk 0.3342498316931352
knoxk 0.48958553751264844
knoxk 0.5371579800525071
knoxk 0.402929789965476
knoxk 0.28874400897181496
knoxk 0.18570064859988372
knoxk 0.2695400774710812
knoxk 0.7023917038728578
knoxk 0.6919020948398497
knoxk 0.34908207350174786
knoxk 0.7171449206240901
knoxk 0.37710141685630943
knoxk 0.16986577939290384
knoxk 0.13421658309995746
knoxk 0.4431056035815755
knoxk 0.5890865078722839
knoxk 0.3823744155326662
knoxk 0.4907591782098676
knoxk 0.47652582717092085
knoxk 0.4343567531298058
knoxk 0.47927032498019884
knoxk 0.41227853349158505
knoxk 0.5624557881275213
knoxk 0.5987052738845337
knoxk 0.7052213745310832
knoxk 0.5583325013563437
knoxk 0.18129726302411145
knoxk 0.6473813390030304
knoxk 0.515119248487882
knoxk 0.5651078968912902
knoxk 0.477162620785266
knoxk 0.3367311407060133
knoxk 0.2833510980143484
knoxk 0.571800543073786
knoxk 0.504673050338951
knoxk 0.3122432037036984
knoxk 0.6239776121

knoxk 0.33130630358733826
knoxk 0.316692707026824
knoxk 0.6496922646257184
knoxk 0.3566596183873621
knoxk 0.2394527725182937
knoxk 0.31443686408820454
knoxk 0.2766667309120254
knoxk 0.3992851068544049
knoxk 0.37569349314052586
knoxk 0.465361488761884
knoxk 0.29803726868239994
knoxk 0.39091156016787604
knoxk 0.5088729222469532
knoxk 0.4995982296303376
knoxk 0.3708200365234176
knoxk 0.22918643187930407
knoxk 0.609715638135619
knoxk 0.4401888992921163
knoxk 0.27103965269908
knoxk 0.5916071186657907
knoxk 0.4637048435073497
knoxk 0.4508142988113936
knoxk 0.36670528625418203
knoxk 0.10461784378599016
knoxk 0.33613456448364254
knoxk 0.6104466709971884
knoxk 0.6292221205061272
knoxk 0.6745954068529098
knoxk 0.3504592887601848
knoxk 0.689391713569267
knoxk 0.30942666101949556
knoxk 0.2643763422397643
knoxk 0.3034843979443769
knoxk 0.7197150633376321
knoxk 0.4071406990111546
knoxk 0.6352941155305443
knoxk 0.2766679216626424
knoxk 0.4668108118750784
knoxk 0.2576798267103471
knoxk 0.6118475039810

knoxk 0.700331085901268
knoxk 0.31890674843859995
knoxk 0.17878650699661103
knoxk 0.5473936225850983
knoxk 0.3903177857892396
knoxk 0.44200191248350756
knoxk 0.4753549500196212
knoxk 0.5542526846636084
knoxk 0.3911683817693609
knoxk 0.2520099822782306
knoxk 0.3623780902727514
knoxk 0.34800409878451977
knoxk 0.3011191587987192
knoxk 0.5852370256004753
knoxk 0.46897504431905523
knoxk 0.5097262980623183
knoxk 0.3224435006815526
knoxk 0.724918355448581
knoxk 0.514451828377649
knoxk 0.4530603658596474
knoxk 0.2504070050190757
knoxk 0.40654230277452275
knoxk 0.2678134581905153
knoxk 0.12560291525273787
knoxk 0.47227235884007895
knoxk 0.3405367345316192
knoxk 0.4999715824222681
knoxk 0.4813481371198685
knoxk 0.46280764370673577
knoxk 0.7252935115041358
knoxk 0.45570650444173416
knoxk 0.37807947321636987
knoxk 0.30898116697546923
knoxk 0.5678150804843589
knoxk 0.4404443902654921
knoxk 0.21940858309563652
knoxk 0.44719825320803225
knoxk 0.4744226856085243
knoxk 0.37867610983944944
knoxk 0.47316

knoxk 0.41171594160595093
knoxk 0.2774980445574495
knoxk 0.2885104584430878
knoxk 0.1267222629768604
knoxk 0.43944855413446393
knoxk 0.280804241624951
knoxk 0.2191619900509836
knoxk 0.34100473357678524
knoxk 0.19604962629419861
knoxk 0.6187397849456339
knoxk 0.3905829058439791
knoxk 0.4449288643652879
knoxk 0.330706933765349
knoxk 0.4237994877128195
knoxk 0.5601044605328335
knoxk 0.6536399186259746
knoxk 0.5983587099434647
knoxk 0.3439431962738997
knoxk 0.300374163291069
knoxk 0.20391665304894702
knoxk 0.6291042347807282
knoxk 0.6349461994516783
knoxk 0.40513969575502606
knoxk 0.5994577012958652
knoxk 0.3344117314394911
knoxk 0.48586766870895587
knoxk 0.4561396186523513
knoxk 0.29228955570828297
knoxk 0.24147804299533035
knoxk 0.4454378203305052
knoxk 0.5703347944077158
knoxk 0.35542417050246955
knoxk 0.3881735450521566
knoxk 0.3743120374776742
knoxk 0.49379447392519377
knoxk 0.5069013890358602
knoxk 0.622008078190759
knoxk 0.4232708952684175
knoxk 0.5967111006504807
knoxk 0.3213193262

In [18]:
print("계산결과값은 " + str(round(np.average(price_si)*10000,0))+"원 입니다.")
print("실제가격은 7,427.46원입니다.")


계산결과값은 8063.0원 입니다.
실제가격은 7,427.46원입니다.
